In [ ]:
import os
import glob
import numpy as np
import tifffile as tiff
import cv2
from skimage.measure import label
from edf import EDF

#   Funciones de Ground Truth
def binarize_and_count(gt_path):
    """Carga GT, binariza todo >0 como célula y cuenta objetos."""
    img = tiff.imread(gt_path)

    if img.ndim == 3:
        img = img[0]  # GT a veces viene como stack

    if img.dtype == np.uint16:
        img = np.clip(img, 0, 255).astype(np.uint8)

    # Binarización simple
    _, binary = cv2.threshold(img, 1, 255, cv2.THRESH_BINARY)

    # Conteo de objetos conectados
    num_labels = label(binary).max()
    return img, binary, num_labels

def gt_to_boxes(gt_binary):
    """Convierte la máscara binaria del GT a cajas bounding boxes."""
    lbl = label(gt_binary)
    boxes = []

    for i in range(1, lbl.max()+1):
        ys, xs = np.where(lbl == i)
        x1, x2 = xs.min(), xs.max()
        y1, y2 = ys.min(), ys.max()
        boxes.append([x1, y1, x2, y2])

    return boxes

#   Funciones para YOLO
def normalize_16bit_to_8bit(img):
    p1, p99 = np.percentile(img, (1, 99))
    img = np.clip(img, p1, p99)
    img = (img - img.min()) / (img.max() - img.min())
    return (img * 255).astype(np.uint8)

def process_for_yolo(image_path):
    img_stack = tiff.imread(image_path)
    original = img_stack[0] if img_stack.ndim == 3 else img_stack
    img_norm = normalize_16bit_to_8bit(original) if original.dtype == np.uint16 else original.copy()
    img_yolo = cv2.cvtColor(img_norm, cv2.COLOR_GRAY2BGR) if img_norm.ndim == 2 else img_norm.copy()
    return original, img_yolo

def draw_boxes(image, boxes):
    img_drawn = image.copy()
    for box in boxes:
        x1, y1, x2, y2 = map(int, box[:4])
        conf = box[4]
        cv2.rectangle(img_drawn, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(img_drawn, f'{conf:.2f}', (x1, y1-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)
    return img_drawn

def filter_boxes_by_area(boxes, factor=2.0):
    """Filtra detecciones cuyo área es mayor que 2 veces el promedio."""
    if len(boxes) == 0:
        return boxes

    # Calcular áreas
    areas = []
    for box in boxes:
        x1, y1, x2, y2 = box[:4]
        areas.append((x2 - x1) * (y2 - y1))

    areas = np.array(areas)
    mean_area = areas.mean()

    # Umbral
    max_allowed = factor * mean_area

    # Filtrar
    filtered = []
    for box, area in zip(boxes, areas):
        if area <= max_allowed:
            filtered.append(box)

    return np.array(filtered)

#   Funciones para TP, FP, FN y score
def point_in_box(x, y, box):
    """Verifica si el punto (x,y) está dentro de la caja."""
    x1, y1, x2, y2 = box[:4]
    return x1 <= x <= x2 and y1 <= y <= y2

def match_boxes(gt_boxes, pred_boxes):
    """Calcula TP, FP, FN según coincidencia de cajas y puntos."""
    matched_pred = set()
    TP = 0

    for gt in gt_boxes:
        y_center = (gt[1] + gt[3]) // 2
        x_center = (gt[0] + gt[2]) // 2
        found = False
        for j, pred in enumerate(pred_boxes):
            if j in matched_pred:
                continue
            if point_in_box(x_center, y_center, pred):
                TP += 1
                matched_pred.add(j)
                found = True
                break
        # Si no se encontró, será FN
    FN = len(gt_boxes) - TP
    FP = len(pred_boxes) - len(matched_pred)
    return TP, FP, FN

def score_image(gt_boxes, pred_boxes):
    TP, FP, FN = match_boxes(gt_boxes, pred_boxes)
    score = TP - FP - FN
    return score, TP, FP, FN

#   Pipeline completo
def process_and_save(image_path, gt_path, model, output_dir):
    # Procesamiento GT
    gt_original, gt_binary, gt_count = binarize_and_count(gt_path)
    gt_boxes = gt_to_boxes(gt_binary)

    # Procesamiento YOLO
    orig, processed = process_for_yolo(image_path)
    boxes = model(processed)
    filtered_boxes = filter_boxes_by_area(boxes, factor=2.0)

    # Score y métricas
    score, TP, FP, FN = score_image(gt_boxes, filtered_boxes)
    print(f'{os.path.basename(image_path)}: Score={score}, TP={TP}, FP={FP}, FN={FN}')

    # Visualización
    img_yolo = draw_boxes(processed, filtered_boxes)
    canvas = np.zeros((orig.shape[0], orig.shape[1]*2, 3), dtype=np.uint8)
    gt_rgb = cv2.cvtColor(gt_binary, cv2.COLOR_GRAY2BGR)
    cv2.putText(gt_rgb, f'GT Count: {gt_count}', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
    canvas[:, :orig.shape[1]] = gt_rgb
    cv2.putText(img_yolo, f'YOLO Det: {len(filtered_boxes)}', (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
    canvas[:, orig.shape[1]:] = img_yolo
    base_name = os.path.splitext(os.path.basename(image_path))[0] + ".jpg"
    cv2.imwrite(os.path.join(output_dir, base_name), canvas)

    return score, TP, FP, FN

if __name__ == "__main__":
    data_dir = "../Fluo-N2DL-HeLa (1)/Fluo-N2DL-HeLa/02"
    gt_dir = "../Fluo-N2DL-HeLa (1)/Fluo-N2DL-HeLa/02_GT/TRA"
    output_dir = "./output/visual/affirmative-0.5"
    os.makedirs(output_dir, exist_ok=True)

    model = EDF('cfg/yolov3.cfg', 'weights/edf', ensemble_option='affirmative', conf_threshold=0.5)

    image_files = sorted(glob.glob(os.path.join(data_dir, "*.tif")))
    gt_files = sorted(glob.glob(os.path.join(gt_dir, "*.tif")))

    scores, tps, fps, fns = [], [], [], []

    for img_path, gt_path in zip(image_files, gt_files):
        print(f'Procesando {os.path.basename(img_path)}')
        score, TP, FP, FN = process_and_save(img_path, gt_path, model, output_dir)
        scores.append(score)
        tps.append(TP)
        fps.append(FP)
        fns.append(FN)

    avg_score = np.mean(scores)

    # Guardar score.txt
    txt_path = os.path.join(output_dir, "score.txt")
    with open(txt_path, "w") as f:
        f.write(f"Average Score: {avg_score:.4f}\n")
        f.write(f"Total TP: {sum(tps)}\n")
        f.write(f"Total FP: {sum(fps)}\n")
        f.write(f"Total FN: {sum(fns)}\n\n")
        f.write("Scores per image:\n")
        for i, s in enumerate(scores):
            f.write(f"Image {i}: Score={s}, TP={tps[i]}, FP={fps[i]}, FN={fns[i]}\n")

    print(f"\nTodas las imágenes y score.txt guardadas en {output_dir}")


Loading checkpoint weights/edf\yolov3-3T3-ckpt.pth
Loading checkpoint weights/edf\yolov3-A-10-ckpt.pth
Loading checkpoint weights/edf\yolov3-A-549-ckpt.pth
Loading checkpoint weights/edf\yolov3-APM-ckpt.pth
Loading checkpoint weights/edf\yolov3-BPAE-ckpt.pth
Loading checkpoint weights/edf\yolov3-CRE-BAG2-ckpt.pth
Loading checkpoint weights/edf\yolov3-CT-ckpt.pth
Loading checkpoint weights/edf\yolov3-CV-1-ckpt.pth
Loading checkpoint weights/edf\yolov3-LLC-MK2-ckpt.pth
Loading checkpoint weights/edf\yolov3-MDBK-ckpt.pth
Loading checkpoint weights/edf\yolov3-MDOK-ckpt.pth
Loading checkpoint weights/edf\yolov3-OK-ckpt.pth
Loading checkpoint weights/edf\yolov3-PL1Ut-ckpt.pth
Loading checkpoint weights/edf\yolov3-RK-13-ckpt.pth
Loading checkpoint weights/edf\yolov3-U2O-S-ckpt.pth
Procesando t000.tif
t000.tif: Score=31, TP=81, FP=7, FN=43
Procesando t001.tif
t001.tif: Score=23, TP=85, FP=15, FN=47
Procesando t002.tif
t002.tif: Score=23, TP=85, FP=13, FN=49
Procesando t003.tif
t003.tif: Score=